# Timing Study

In [3]:
# Import required package
using DifferentialEquations
using COBREXA
using DataFrames
using Tulip
using Plots
using Colors
using ModelingToolkit

function glucaric_acid_f6p(du, u, p, t)
    g6p, f6p, mi, ino1, miox = u

    A, W, v_in, lam = p

    n_ino1, theta_ino1, k_ino1 = W[1]
    n_miox, theta_miox, k_miox = W[2]
    k_ino1 = k_ino1 * 60 * 60
    k_miox = k_miox * 60 * 60

    v_pgi = reversible_michaelismenten(g6p, f6p, ga_params("vm_pgi"), ga_params("keq_pgi"), ga_params("km_pgi_g6p"), ga_params("km_pgi_f6p"))
    v_zwf = michaelismenten(g6p, ga_params("vm_zwf"), ga_params("km_zwf_g6p"))
    v_pfk = hillequation(f6p, ga_params("vm_pfk"), ga_params("n_pfk"), ga_params("km_pfk_f6p"))
    v_ino1 = ino1 * michaelismenten(g6p, ga_params("vm_ino1"), ga_params("km_ino1_g6p"))
    v_tm = michaelismenten(mi, ga_params("vm_t_mi"), ga_params("km_t_mi"))
    v_miox = miox * michaelismenten_substrateactivation(mi, ga_params("vm_miox"), ga_params("km_miox_mi"), ga_params("a_miox"), ga_params("ka_miox_mi"))

    u_ino1_mi = sum(A[1] .* [activation(mi, k_ino1, theta_ino1, n_ino1), repression(mi, k_ino1, theta_ino1, n_ino1), k_ino1])
    u_miox_mi = sum(A[2] .* [activation(mi, k_miox, theta_miox, n_miox), repression(mi, k_miox, theta_miox, n_miox), k_miox])

    du[1] = v_in - v_zwf - v_pgi - v_ino1 - lam*g6p
    du[2] = v_pgi + 0.5*v_zwf - v_pfk - lam*f6p
    du[3] = v_ino1 - v_tm - v_miox - lam*mi
    du[4] = u_ino1_mi  - lam*ino1
    du[5] = u_miox_mi - lam*miox
end

function ga_params(param_name)
    params = Dict("vm_ino1" => 0.2616 * 60 * 60, #[mM/s],
    "km_ino1_g6p" => 1.18, #[mM],
    "vm_t_mi"=> 0.045  * 60 * 60, #[mM/s],
    "km_t_mi"=> 15, #[mM],
    "vm_miox" => 0.2201 * 60 * 60, #[mM/s],
    "km_miox_mi" => 24.7, #[mM],
    "a_miox" => 5.4222, #[no units],
    "ka_miox_mi" => 20, #[mM],
    "vm_glc" => 0.1 * 60 * 60,#EC 2.7.1.63 [mM/s],
    "km_glc" => 0.082, #[mM]
    "vm_pgi" => 1.1094 * 60 * 60,#[mM/s],
    "keq_pgi" => 0.3, #[no units],
    "km_pgi_g6p" => 0.28, #[mM],
    "km_pgi_f6p" => 0.147, #[mM],
    "vm_pfk" => 2.615 * 60 * 60,#[mM/s],
    "km_pfk_f6p" => 0.16, #[mM],
    "n_pfk" => 3, #[no units],
    "vm_zwf" => 0.0853 * 60 * 60,
    "km_zwf_g6p" => 0.1,
    "v_pts" => 0.1656 * 60 * 60,
    "kcat_udh" => 161 * 60 * 60, #[1/s], from Brenda
    "km_udh" => 0.2 #[mM], from Brenda
    )
    return params[param_name]
end

repression(x, k, theta, n) = k/(1+(x/theta)^n)
activation(x, k, theta, n) = (k*(x^n))/(theta^n + x^n)
reversible_michaelismenten(x, y, vm, keq, kmx, kmy) = (vm*(x - (y/keq)))/(x + kmx*(1+(y/kmy)))
hillequation(x, vm, n, km) = vm*x^n / (km^n + x^n)
michaelismenten(x, vm, km) = (vm*x)/(km+x)
michaelismenten_substrateactivation(x, vm, km, a, ka) = ((vm * (1+ (a*x)/(ka + x)))*x)/(km + x)


michaelismenten_substrateactivation (generic function with 1 method)

In [9]:
function timing_study_ga(N)
    #Comparison to ML surrogate GA model
    #Specify algorithm parameters
    deltat = 1/(60*60) #genetic timescale, seconds
    A = [[0, 1, 0], [1, 0, 0]]
    W = [[2., 0.013319, 0.608497], [2., 8.013213, 0.001996]] #Optimal from MSc work
    #W = [[2., 10., 0.608497], [2., 8.013213, 0.001996]]
    #Run pre-simulation with initial FBA v_in, v_out lam
    model = load_model(StandardModel, "../models/iML1515.xml") #load core E.coli model

    #Modify model to add dynamic pathway influx 
    reaction = Reaction("DynPathway", Dict("M_g6p_c" => -1.0))
    reaction.lb = 0.0 #Must include lower bound to avoid reaction running backwards with -1000 flux
    add_reactions!(model, [reaction])

    v_dp = 0.0
    restricted_fluxes = flux_balance_analysis_dict(model, Tulip.Optimizer, modifications = [change_constraint("DynPathway", lb = v_dp, ub = v_dp)])
    lam = restricted_fluxes["R_BIOMASS_Ec_iML1515_core_75p37M"]
    v_in =  restricted_fluxes["R_TRE6PH"] + restricted_fluxes["R_PGMT"] + restricted_fluxes["R_HEX1"] + restricted_fluxes["R_AB6PGH"] + restricted_fluxes["R_AB6PGH"] - restricted_fluxes["R_TRE6PS"] + restricted_fluxes["R_FRULYSDG"] + restricted_fluxes["R_GLCptspp"] - restricted_fluxes["R_G6PP"] + restricted_fluxes["R_G6Pt6_2pp"] + restricted_fluxes["R_BGLA1"] 
    #println("ODE initial parameters: ", [v_in, lam])

    u0 = [0.281, 0.0605, 0., 0., 0.] #From Verma paper
    #println("ODE initial conditions: ", u0)

    #Redefine param vector to include decision variables
    p = [A, W, v_in, lam]

    #instantiate initial times
    starttime = 0.
    endtime = starttime + deltat
    tspan = [starttime, endtime]
    savetimes = [starttime, endtime] #saving start time and end times for alternate v_dp calculation
    u0 = [0.281, 0.0605, 0., 0., 0.]

    #FBA-ODE optimization loop
    ode_data = DataFrame("time" => [0], "g6p" => u0[1], "f6p" => u0[2], "mi" => u0[3], "ino1" => u0[4], "miox" => u0[5], "v_dp" => [0], "v_out" => restricted_fluxes["R_PFK"], "flag" => [0])
    #ode_data = DataFrame()
    fba_data = DataFrame("time" => [0], "v_in" => [v_in], "lam" => [lam]);

    total_ode_time = 0.0
    total_fba_time = 0.0
    #println("Beginning loop...")
    for i in 1:N
        # println("Iteration ", i)
        prob = ODEProblem(glucaric_acid_f6p, u0, tspan, p)
        #Solve ODE
        total_ode_time += @elapsed begin
            sol = solve(prob, Rosenbrock23(), reltol=1e-3, abstol=1e-6, saveat=savetimes)
        end
        #Solve for pathway fluxes from final concentrations
        v_dp = sol.u[end][4] * michaelismenten(sol.u[end][1], ga_params("vm_ino1"), ga_params("km_ino1_g6p")) #same as v_ino1 = ino1*mm(g6p, vm, km)
        #v_dp = (sol.u[end][3] - sol.u[1][3])/(sol.t[end] - sol.t[1]) #alternate tangent method of computing v_dp
        #println("Dynamic pathway flux prediction from ODE: ", v_dp)
        v_out = hillequation(sol.u[end][2], ga_params("vm_pfk"), ga_params("n_pfk"), ga_params("km_pfk_f6p"))
        #println("PFK flux prediction from ODE: ", v_out)

        
        #Backstop to loop
        if v_dp >= v_in
            ode_data = vcat(ode_data, DataFrame("time" => sol.t[1], "g6p" => sol.u[1][1], "f6p" => sol.u[1][2], "mi" => sol.u[1][3], "ino1" => sol.u[1][4], "miox" => sol.u[1][5], "v_dp" => v_dp, "v_out" => v_out, "flag" => [1]))
            v_dp = 8.5
            v_out = 0.5
        else    ode_data = vcat(ode_data, DataFrame("time" => sol.t[1], "g6p" => sol.u[1][1], "f6p" => sol.u[1][2], "mi" => sol.u[1][3], "ino1" => sol.u[1][4], "miox" => sol.u[1][5], "v_dp" => v_dp, "v_out" => v_out, "flag" => [1]))
        end

        #Solve restricted FBA
        #restricted_fluxes = flux_balance_analysis_dict(model, Tulip.Optimizer, modifications = [change_constraint("DynPathway", lb = v_dp, ub = v_dp)]) #No Vout integration
        total_fba_time += @elapsed begin
            restricted_fluxes = flux_balance_analysis_dict(model, Tulip.Optimizer, modifications = [change_constraint("DynPathway", lb = v_dp, ub = v_dp), change_constraint("R_PFK", lb = v_out, ub = v_out)])
        #println("Dynamic pathway flux prediction from FBA: ", restricted_fluxes["DynPathway"])
        end
        #Update parameter and timing values
        lam = restricted_fluxes["R_BIOMASS_Ec_iML1515_core_75p37M"]
        v_in =  restricted_fluxes["R_TRE6PH"] + restricted_fluxes["R_PGMT"] + restricted_fluxes["R_HEX1"] + restricted_fluxes["R_AB6PGH"] + restricted_fluxes["R_AB6PGH"] - restricted_fluxes["R_TRE6PS"] + restricted_fluxes["R_FRULYSDG"] + restricted_fluxes["R_GLCptspp"] - restricted_fluxes["R_G6PP"] + restricted_fluxes["R_G6Pt6_2pp"] + restricted_fluxes["R_BGLA1"] 
        
        p = [A, W, v_in, lam]

        starttime = endtime
        endtime = starttime + deltat
        tspan = [starttime, endtime]
        savetimes = [starttime, endtime]

        u0 = sol.u[end]
        #Save FBA data
        fba_data = vcat(fba_data, DataFrame("time" => [starttime], "v_in" => [p[3]], "lam" => [p[4]]))
    end
    println("Number of iterations ", N)
    println("Total FBA time ", total_fba_time)
    println("Total ODE time ", total_ode_time)
    println("Fraction of FBA time ", total_fba_time/(total_ode_time+total_fba_time))
end

timing_study_ga (generic function with 1 method)

In [ ]:
@time timing_study_ga(10)

In [11]:
@time timing_study_ga(100)

Number of iterations 100
Total FBA time 27.4331191
Total ODE time 0.6012364
Fraction of FBA time 0.9785535857958282
 31.231104 seconds (55.15 M allocations: 30.399 GiB, 7.31% gc time)


In [12]:
@time timing_study_ga(1000)

Number of iterations 1000
Total FBA time 270.97098519999975
Total ODE time 4.606375600000002
Fraction of FBA time 0.9832846370738594
279.028147 seconds (524.88 M allocations: 297.844 GiB, 8.37% gc time)


In [13]:
@time timing_study_ga(3600)

Number of iterations 3600
Total FBA time 1005.3663223000015
Total ODE time 12.430016599999993
Fraction of FBA time 0.9877873243153596
1022.374149 seconds (1.88 G allocations: 1.053 TiB, 8.16% gc time)
